In [122]:
import pandas as pd
import re

In [1]:
import os
from dotenv import load_dotenv

SA_KEY=os.getenv("GOOGLE_SA_KEY")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SA_KEY

In [56]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print("Texts:")

    for text in texts:
        print(f'\n"{text.description}"')

        vertices = [
            f"({vertex.x},{vertex.y})" for vertex in text.bounding_poly.vertices
        ]

        print("bounds: {}".format(",".join(vertices)))

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return response

In [57]:
response = detect_text("/Users/tobiaspoetzl/coding/creamCheese_backup/images/Rewe_Bons_Scans_ToP/Rewe_T09.jpg")

Texts:

"REWE
Ziegelstrasse 232
23556 Lübeck
Tel.: 0451 - 89080
UID Nr.: DE812706034
SPITZKOHL
1,012 kg x 1,99 EUR/kg
BANANE BIO
0,614 kg x 1,99 EUR/kg
COMBIPLUS ZITRUS
Treuerabatt
MAISHAEHNCHEN
DELI. BACON
2 Stk x 1,29
JA RUTTERKAESE
GO JUNG
MOZZARELLA 45%
4 Stk x 0,99
RICOTTA
FRISCHK. KRAEUTE
EIWEISSBROT
EIWEISS TOASTBR.
HIGHPROTEINTOAST
LACHSFORELLE
2 Stk x 2,99
LIMETTE
MANGO
2,59
AVOCADO FEINE W.
3 Stk x
BLUMENKOHL WEISS
BABYSPINAT
SALAT ROMA HERZ
3 Stk x 1,99
RUCOLA
KAROTTE M. GRUEN
2 Stk x 1,29
BOHNE BUSCH
BASILIKUM
PETERSILIE KRAUS
EIE 3H M-L
BUTTERMILCH
SKYR NATUR
2 Stk x
SKYR NATUR
2 Stk x
GARNELE NATUR
CENTER
0,99
1,89
2 Stk x 3,69
JA! RAHMSPINAT
TOM KHA GAI
INST. NUD. TOM YUM
2 Stk x
BALSAMICO CREME
1,19
1,29
KOKOSMILCH
2 Stk x
THAI CURRY PASTE
ALTOBELLO BIANCO
SYRAH ROSE
SCHW. JOHANNISB
PFAND 0,25 EURO
RB BIO LIMETTE
HENKELL ROSE
2 Stk x 3,99
EARL GRAY TEE
GESCHIRRTABS
KRAFTREIN. FETTL.
BACK, -GRILLREIN.
ELMEX ZAHNP, PROF
HANDCREME KNEIPP
LIPGLOSS 102
SUMME
Geg. VISA
Datum:

In [121]:
end = len(response.text_annotations) + 1
texts = response.text_annotations[1:end]

# Erstellen eines leeren DataFrames mit den gewünschten Spaltennamen
columns = ["String", "x1", "x2", "x3", "x4", "y1", "y2", "y3", "y4"]
df_receipt = pd.DataFrame(columns=columns)

for i, text in enumerate(texts):
    #print(f'\n"{text.description}"')
    df_receipt.loc[i, "String"] = text.description
    vertices = [
        f"({vertex.x},{vertex.y})" for vertex in text.bounding_poly.vertices
    ]

    for vertex in text.bounding_poly.vertices:
        #print(vertex)
        # Extrahieren der x- und y-Werte
        for j in range(4):
            df_receipt.iloc[i, j+1] = text.bounding_poly.vertices[j].x
            df_receipt.iloc[i, j+5] = text.bounding_poly.vertices[j].y



print(df_receipt[10:20])

         String   x1   x2   x3   x4   y1   y2   y3   y4
10          UID  217  278  278  217  533  533  565  565
11           Nr  296  334  334  296  533  533  565  565
12           .:  339  374  374  339  533  533  565  565
13  DE812706034  397  613  613  397  533  534  567  566
14    SPITZKOHL   38  217  217   38  612  612  646  646
15        1,012   79  179  179   79  653  653  690  690
16           kg  197  239  239  197  653  653  690  690
17            x  255  278  278  255  653  653  690  690
18         1,99  338  418  418  338  653  653  690  690
19          EUR  438  497  497  438  653  653  690  690


In [6]:
detect_text("/Users/tobiaspoetzl/coding/creamCheese_backup/images/Rewe_Bons_Scans_ToP/Rewe_T01.jpg")

Texts:

"ZIEGELSTR.
23556 LÜBECK
Tel. 0451 48065836
UID Nr.: DE277457805
JA! BUTTERKAESE
SALAT LOLLO BION
PICCOLIN. BREZEL
2 Stk x
LAUGENSTANGEN
3,49
2 Stk x 1,99
BUTTERCROISSANTS
NUTELLA 450G
CHIPSFRISCH CHIL
LIMONCELLO
1,09
0, 15
LOSCHER CLUBMATE
3 Stk x
PFAND 0, 15 EUR
3 Stk x
ROTK. ALKOHOLFREI
GLUECKSSCHWEIN
4 Stk x 0,80
FOLIENBALLONS
GIRLANDE DEKO
KONFETTI
16X FORM-SERVIET
WACHSGIESSEN
Wachsgiessen 2, 69
Funny Frisch 0,89
SUMME
Geg. VISA
Datum:
EUR
EUR
* * Kundenbeleg * *
EUR
2,79 B
2,29 B
6,98 B
3,98 B
3,69 B
3, 29 B
0,99 B
13, 79 A
3,27 A
0,45 A *
4,99 A
3,20 B
3,99 A
3,99 A
2,49 A
2,49 A
2,99 A
-0,30 A
-0,10 B
65,26
65,26
30.12.2023"
bounds: (61,0),(1655,0),(1655,2787),(61,2787)

"ZIEGELSTR"
bounds: (466,2),(803,2),(803,61),(466,61)

"."
bounds: (811,2),(824,2),(824,61),(811,61)

"23556"
bounds: (580,65),(767,64),(767,126),(580,127)

"LÜBECK"
bounds: (806,64),(1034,63),(1034,125),(806,126)

"Tel"
bounds: (390,139),(491,138),(491,201),(390,202)

"."
bounds: (502,139),(522,139),(

In [8]:
detect_text("/Users/tobiaspoetzl/coding/creamCheese_backup/images/Rewe_Bons_Scans_ToP/Rewe_T18.jpg")

Texts:

"REWE MARKT Grühn oHG
ZIEGELSTR. 7 - 13
23556 LÜBECK
Tel.: 0451 - 48065836
UID Nr.: DE277457805
SENF MITTELSCH.
TOMATENMARK
TOM.MARK KONZ.
GEHACKTE TOMATEN
3 Stk x 0,39
SFB PH86 DP
TINTEN.DESCHER
SENSEO SWITCH
SUMME
Geg EWE Guthaben
Restbetrag
Geg. VISA
Datum:
Uhrzeit:
EUR
EUR
EUR
EUR
0,49 B
0,49 B
1,49 B
1,17 B
* * Kundenbeleg * *
9,99 A
2,99 A
66,00 A
82,62
10,00
72,62
72,62
23.04.2018
20:03:31 Uhr"
bounds: (50,234),(1199,234),(1199,1977),(50,1977)

"REWE"
bounds: (258,241),(456,239),(456,289),(258,291)

"MARKT"
bounds: (486,239),(632,238),(632,288),(486,289)

"Grühn"
bounds: (660,238),(803,237),(803,287),(660,288)

"oHG"
bounds: (833,237),(919,236),(919,285),(833,286)

"ZIEGELSTR"
bounds: (343,309),(602,306),(603,360),(344,363)

"."
bounds: (610,307),(624,307),(625,360),(611,360)

"7"
bounds: (659,306),(687,306),(688,359),(660,359)

"-"
bounds: (718,306),(745,306),(746,359),(719,359)

"13"
bounds: (774,305),(835,304),(836,357),(775,358)

"23556"
bounds: (428,382),(571,380),(

"1,17"
bounds: (1004,938),(1118,938),(1118,991),(1004,991)

"GEHACKTE"
bounds: (56,943),(281,942),(281,994),(56,995)

"TOMATEN"
bounds: (310,942),(513,941),(513,993),(310,994)


"1,49"
bounds: (1005,872),(1118,870),(1119,921),(1006,923)

"TOM.MARK"
bounds: (56,873),(280,872),(280,925),(56,926)

"KONZ"
bounds: (310,871),(425,870),(425,923),(310,924)

In [9]:
detect_text("/Users/tobiaspoetzl/coding/wesplit-receipt-ocr/raw/IMG_4855.jpg")

Texts:

"Netto
Marken-Discount
6772 Zschornewitz, Straße des Friedens
WWW.NETTO-ONLINE.DE
KN Toasties sort. 335g
KN Toasties sort. 335g
Bisc. Schokobroetchen360g
Rue.veg.SchinkenspickerPap.809
Rabatt 30%
Naggi GA Gespenstersuppe 750ml
Bio BB Kontepulciano0,75L
Liebl. Sahnepudding sort. 500g
Clarkys Kesselchips sort. 150g
Spitzkohl
Sellerie
0.420 kg x 1.19 EUR/kg
Feldsalat 150g
Preisaenderung
Paprika rot 500g
2 X
Gurken
Lauchzwiebel
SUNNE [15]
SUMME
Bar EUR
A 19%
P
7%
MWST
0.45
1.15
€
ㅁ
1.19
Struer
-EUR
1,19 B
1.19 B
1,89 B
1,59 B
BRUTTO
2.79
17.64
-0.48
0,99 B
2,79 A
1,39 B
1.59 B
1.99 B
0,50 B
1.49 B
-0.75
NETTO
2,34
16.49
Nit der DeutschlandCard hätten Sie
10 Punkte erhalten!
1.89 B
2.38 B
0,79 B
20.43 -
20.43
20.43
TSE Transaktionsnummer.: 359242
Seriennr. Kasse:NTO-000-04868-3
Prüfuert:nYiITDCCGZQ/zVyvne0jsetRvU9xbCe
xizOPClazosMgPtBivSda5C5pCUKiih834Z1+CHG
le0nkcWtcBMyPS23N0e2041 Byqq0jWJcZN+13S/a
4e6eekD0gj8UI9Dvq
Signaturzähler:809479
TSE-Start:2023-11-27715:21:16.000Z
TSE-Stop 